In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

--2025-04-21 03:38:24--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85497 (83K) [text/plain]
Saving to: ‘train_dreambooth_lora_sdxl.py’

train_dreambooth_lo 100%[===================>]  83.49K  --.-KB/s    in 0.002s  

2025-04-21 03:38:25 (42.9 MB/s) - ‘train_dreambooth_lora_sdxl.py’ saved [85497/85497]



In [1]:
!pip install -q diffusers==0.24.0 accelerate transformers xformers peft safetensors
!pip install -q git+https://github.com/huggingface/diffusers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  G

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip freeze

absl-py==1.4.0
accelerate==1.5.2
aiohappyeyeballs==2.6.1
aiohttp==3.11.15
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.5
ale-py==0.10.2
altair==5.5.0
annotated-types==0.7.0
anyio==4.9.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.1
arviz==0.21.0
astropy==7.0.1
astropy-iers-data==0.2025.4.14.0.37.22
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
backports.tarfile==1.2.0
beautifulsoup4==4.13.4
betterproto==2.0.0b6
bigframes==1.42.0
bigquery-magics==0.9.0
bleach==6.2.0
blinker==1.9.0
blis==1.3.0
blosc2==3.3.0
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.89
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.21.0
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confe

In [ ]:
%ls

data/  drive/  sample_data/  train_dreambooth_lora_sdxl.py


In [ ]:
%mkdir data/captions

In [ ]:
import os

MODEL_NAME = "stabilityai/stable-diffusion-xl-base-1.0"
OUTPUT_DIR = "./sdxl-dreambooth-output"
DATA_DIR = "./data"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
from IPython.display import display
from google.colab import files

# Convert to DreamBooth format
import shutil
from pathlib import Path

dreambooth_dir = "./dreambooth-directory"
Path(dreambooth_dir).mkdir(exist_ok=True)

images_path = Path(DATA_DIR) / "images"
captions_path = Path(DATA_DIR) / "captions"

i = 0
for img_file in sorted(images_path.glob("*.jpg")):
    base_name = img_file.stem
    caption_file = captions_path / f"{base_name}.txt"
    if caption_file.exists():
        dest_img = Path(dreambooth_dir) / f"{i:04d}.jpg"
        dest_txt = Path(dreambooth_dir) / f"{i:04d}.txt"
        shutil.copy(img_file, dest_img)
        shutil.copy(caption_file, dest_txt)
        i += 1

print(f"✅ Prepared {i} image-caption pairs for DreamBooth.")



✅ Prepared 103 image-caption pairs for DreamBooth.


In [ ]:
from datasets import Dataset
from PIL import Image
import pandas as pd

image_dir = "./dreambooth-directory"
data = []

for img_file in sorted(Path(image_dir).glob("*.jpg")):
    caption_file = img_file.with_suffix(".txt")
    if caption_file.exists():
        data.append({
            "image": str(img_file),  # image path
            "caption": caption_file.read_text().strip()
        })

df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

# Save the dataset
dataset_path = "./dreambooth_hf_dataset"
dataset.save_to_disk(dataset_path)

# from datasets import Dataset, DatasetDict, load_from_disk
# from PIL import Image
# import pandas as pd
# from pathlib import Path

# image_dir = "./dreambooth-directory"
# data = []

# for img_file in sorted(Path(image_dir).glob("*.jpg")):
#     caption_file = img_file.with_suffix(".txt")
#     if caption_file.exists():
#         data.append({
#             "image": str(img_file),  # image path
#             "caption": caption_file.read_text().strip()
#         })

# df = pd.DataFrame(data)
# train_dataset = Dataset.from_pandas(df)

# # Create a DatasetDict with a train split
# dataset_dict = DatasetDict({"train": train_dataset})

# # Save the dataset
# dataset_path = "./dreambooth_hf_dataset"
# dataset_dict.save_to_disk(dataset_path)

# # Verify the structure
# test_load = load_from_disk(dataset_path)
# print(test_load)
# # print("Keys:", list(test_load.keys()))
# print("Train columns:", test_load["train"].column_names)

Saving the dataset (0/1 shards):   0%|          | 0/102 [00:00<?, ? examples/s]

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 33.0 MB/s eta 0:00:00


In [ ]:
dataset = Dataset.load_from_disk(dataset_path)

In [ ]:
print("Train columns:", test_load.column_names)

Train columns: ['image', 'caption']


In [ ]:
print(dataset['image'])

['dreambooth-directory/0000.jpg', 'dreambooth-directory/0001.jpg', 'dreambooth-directory/0002.jpg', 'dreambooth-directory/0003.jpg', 'dreambooth-directory/0004.jpg', 'dreambooth-directory/0005.jpg', 'dreambooth-directory/0006.jpg', 'dreambooth-directory/0007.jpg', 'dreambooth-directory/0008.jpg', 'dreambooth-directory/0009.jpg', 'dreambooth-directory/0010.jpg', 'dreambooth-directory/0011.jpg', 'dreambooth-directory/0012.jpg', 'dreambooth-directory/0013.jpg', 'dreambooth-directory/0014.jpg', 'dreambooth-directory/0015.jpg', 'dreambooth-directory/0016.jpg', 'dreambooth-directory/0017.jpg', 'dreambooth-directory/0018.jpg', 'dreambooth-directory/0019.jpg', 'dreambooth-directory/0020.jpg', 'dreambooth-directory/0021.jpg', 'dreambooth-directory/0022.jpg', 'dreambooth-directory/0023.jpg', 'dreambooth-directory/0024.jpg', 'dreambooth-directory/0025.jpg', 'dreambooth-directory/0026.jpg', 'dreambooth-directory/0027.jpg', 'dreambooth-directory/0028.jpg', 'dreambooth-directory/0029.jpg', 'dreamboo

In [ ]:
# Training setup
from diffusers import StableDiffusionXLPipeline
from pathlib import Path

!accelerate config default


!accelerate launch train_dreambooth_lora_sdxl.py \
--pretrained_model_name_or_path=$MODEL_NAME \
--dataset_name=./dreambooth_hf_dataset \
--image_column=image \
--caption_column=caption \
--output_dir=$OUTPUT_DIR \
--resolution=1024 \
--train_batch_size=1 \
--gradient_accumulation_steps=4 \
--learning_rate=1e-5 \
--lr_scheduler="constant" \
--use_8bit_adam \
--max_train_steps=3000 \
--checkpointing_steps=500 \
--enable_xformers_memory_efficient_attention \
--mixed_precision=fp16 \
--instance_prompt="a surreal collage representing sonic identity"


Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.
2025-04-21 05:15:09.356674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745212509.378501   27508 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745212509.385043   27508 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
04/21/2025 05:15:14 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type

In [ ]:
!zip -r fine_tuned_sdxl.zip sdxl-dreambooth-output/

  adding: sdxl-dreambooth-output/ (stored 0%)
  adding: sdxl-dreambooth-output/checkpoint-1000/ (stored 0%)
  adding: sdxl-dreambooth-output/checkpoint-1000/random_states_0.pkl (deflated 25%)
  adding: sdxl-dreambooth-output/checkpoint-1000/scaler.pt (deflated 60%)
  adding: sdxl-dreambooth-output/checkpoint-1000/optimizer.bin (deflated 11%)
  adding: sdxl-dreambooth-output/checkpoint-1000/scheduler.bin (deflated 55%)
  adding: sdxl-dreambooth-output/checkpoint-1000/pytorch_lora_weights.safetensors (deflated 6%)
  adding: sdxl-dreambooth-output/checkpoint-3000/ (stored 0%)
  adding: sdxl-dreambooth-output/checkpoint-3000/random_states_0.pkl (deflated 25%)
  adding: sdxl-dreambooth-output/checkpoint-3000/scaler.pt (deflated 60%)
  adding: sdxl-dreambooth-output/checkpoint-3000/optimizer.bin (deflated 11%)
  adding: sdxl-dreambooth-output/checkpoint-3000/scheduler.bin (deflated 55%)
  adding: sdxl-dreambooth-output/checkpoint-3000/pytorch_lora_weights.safetensors (deflated 6%)
  adding: 

In [ ]:
from google.colab import files
files.download('fine_tuned_sdxl.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>